<a href="https://colab.research.google.com/github/RiturajSaha/Malaria-Detector-Application/blob/master/MD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importng Libraries

In [0]:
import numpy as np 
import pandas as pd
import os 
import cv2
import matplotlib.pyplot as plt 
import os
from PIL import Image
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.utils import np_utils
from tensorflow.python.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,GlobalAveragePooling2D,Activation,BatchNormalization,Dropout
from tensorflow.python.keras import Sequential,backend,optimizers

Using TensorFlow backend.


Unzipping Dataset

In [0]:
from zipfile import ZipFile
file_name='/content/drive/My Drive/Colab Notebooks/mldataset.zip'
with ZipFile(file_name,'r') as zip:
          zip.extractall()

Converting Images to Array

In [0]:
data = []
labels = []

for img in parasitized_data:
    try:
        img_read = plt.imread('/content/mldataset/Parasitized/' + "/" + img)
        img_resize = cv2.resize(img_read, (50, 50))
        img_array = img_to_array(img_resize)
        data.append(img_array)
        labels.append(1)
    except:
        None
        
for img in uninfected_data:
    try:
        img_read = plt.imread('/content/mldataset/Uninfected' + "/" + img)
        img_resize = cv2.resize(img_read, (50, 50))
        img_array = img_to_array(img_resize)
        data.append(img_array)
        labels.append(0)
    except:
        None

image_data = np.array(data)
labels = np.array(labels)

Dividng into train and test

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(image_data, labels, test_size = 0.2, random_state = 0)

y_train = np_utils.to_categorical(y_train, num_classes = 2)
y_test = np_utils.to_categorical(y_test, num_classes = 2)

Building the CNN model

In [0]:
height, width, classes, channels=50,50,2,3
model = Sequential()
    
inputShape = (height, width, channels)
chanDim = -1
    
if backend.image_data_format() == 'channels_first':
          inputShape = (channels, height, width)
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape = inputShape))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization(axis = chanDim))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization(axis = chanDim))
model.add(Dropout(0.2))

model.add(Conv2D(32, (3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization(axis = chanDim))
model.add(Dropout(0.2))

model.add(Flatten())
    
model.add(Dense(512, activation = 'relu'))
model.add(BatchNormalization(axis = chanDim))
model.add(Dropout(0.5))
model.add(Dense(classes, activation = 'softmax'))
    
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 24, 24, 32)        128       
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 11, 11, 32)        1

Compiling the model

In [0]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

Fitting the model

In [0]:
model.fit(x_train, y_train, epochs = 25, batch_size = 32)

Epoch 1/25
689/689 [==============================] - 68s 99ms/step - loss: 0.5061 - accuracy: 0.7930
Epoch 2/25
689/689 [==============================] - 68s 99ms/step - loss: 0.2120 - accuracy: 0.9246
Epoch 3/25
689/689 [==============================] - 68s 99ms/step - loss: 0.1803 - accuracy: 0.9374
Epoch 4/25
689/689 [==============================] - 69s 100ms/step - loss: 0.1718 - accuracy: 0.9408
Epoch 5/25
689/689 [==============================] - 71s 103ms/step - loss: 0.1628 - accuracy: 0.9458
Epoch 6/25
689/689 [==============================] - 69s 100ms/step - loss: 0.1553 - accuracy: 0.9484
Epoch 7/25
689/689 [==============================] - 69s 100ms/step - loss: 0.1555 - accuracy: 0.9498
Epoch 8/25
689/689 [==============================] - 69s 99ms/step - loss: 0.1533 - accuracy: 0.9507
Epoch 9/25
689/689 [==============================] - 68s 99ms/step - loss: 0.1513 - accuracy: 0.9519
Epoch 10/25
689/689 [==============================] - 68s 99ms/step - loss: 0

Evaluating the model

In [0]:
predict = model.evaluate(x_test, y_test)
print("Loss: ",predict[0])
print("Accuracy: ",predict[1]*100)

173/173 [==============================] - 5s 27ms/step - loss: 0.1457 - accuracy: 0.9521
Loss:  0.14572061598300934
Accuracy:  95.21045088768005


Saving model

In [0]:
#model.save('a95e25model.h5')
%cd /content/drive/My Drive/Colab Notebooks/
from tensorflow.keras.models import load_model
model = load_model('a95e25model.h5')
model.summary()


/
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 24, 24, 32)        128       
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 11, 11, 32)       